In [1]:
from ultralytics import YOLO
from PIL import Image
import cv2


In [2]:
model = YOLO("Yolo/yolov8n.pt")

In [5]:
model("images/ho_nap/z4203565180843_48458347548433ba59d556e8f0b0752f.jpg", save = True,  save_txt=True)


image 1/1 E:\do-an\images\ho_nap\z4203565180843_48458347548433ba59d556e8f0b0752f.jpg: 640x480 1 vase, 133.0ms
Speed: 2.0ms preprocess, 133.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2
1 label saved to runs\detect\predict2\labels


[ultralytics.yolo.engine.results.Results object with attributes:
 
 _keys: ('boxes', 'masks', 'probs')
 boxes: ultralytics.yolo.engine.results.Boxes object
 keys: ['boxes']
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: '

In [7]:
import tensorflow as tf
import numpy as np
from src.training.load_model import load_model
from src.training.load_image import get_dataset

checkpoint_dir = "runs/training/"
# Load model 
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest
# Create a new model instance
model = load_model(2)

# Load the previously saved weights
model.load_weights(latest)


In [8]:

from glob import glob
testing_image = glob("./images/ho_nap/*.jpg")
for path in testing_image:
    # Evaluate
    img = tf.keras.utils.load_img(
        path, target_size=(244, 244)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    print(np.max(img_array[0]), np.min(img_array[0]))
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    # _,_,class_names = get_dataset()

    class_names = ['chuan', 'thieu_nhan_dan']
    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )

255.0 0.0
1/1 [==============================] - 1s 1s/step
This image most likely belongs to chuan with a 97.98 percent confidence.
255.0 0.0
1/1 [==============================] - 0s 26ms/step
This image most likely belongs to chuan with a 97.98 percent confidence.
255.0 0.0
1/1 [==============================] - 0s 22ms/step
This image most likely belongs to chuan with a 97.98 percent confidence.
255.0 0.0
1/1 [==============================] - 0s 29ms/step
This image most likely belongs to chuan with a 97.98 percent confidence.
255.0 0.0
1/1 [==============================] - 0s 25ms/step
This image most likely belongs to chuan with a 97.98 percent confidence.
255.0 0.0
1/1 [==============================] - 0s 20ms/step
This image most likely belongs to chuan with a 97.98 percent confidence.
255.0 0.0
1/1 [==============================] - 0s 21ms/step
This image most likely belongs to chuan with a 97.98 percent confidence.
255.0 0.0
1/1 [==============================] - 0s 21ms/

In [53]:
train_img, test_img ,_ =get_dataset()

Found 60 files belonging to 2 classes.
Using 48 files for training.
Found 60 files belonging to 2 classes.
Using 12 files for validation.
['chuan', 'thieu_nhan_dan']
0.0 1.0


In [55]:
data,_  = next(iter(train_img)) 